In [1]:
import numpy as np
import matplotlib.pyplot as plt
import  matplotlib.animation as animation
plt.rc('figure', figsize=(12,9))

plt.rcParams['animation.writer'] = "avconv"

# Constantes du système
g, m1, m2, r1, r2 = 9.8, 2, 1, 1, 2

# flux pour les angles
def fluxth(th1, th2, p1, p2):
    c1 = (m2*r2**2*p1-m2*r1*r2*np.cos(th1-th2)*p2)/(m2*r1**2*r2**2*(m1+m2*(np.sin(th1-th2))**2))
    c2 = ((m1+m2)*r1**2*p2-m2*r1*r2*np.cos(th1-th2)*p1)/(m2*r1**2*r2**2*(m1+m2*(np.sin(th1-th2))**2))
    return c1, c2

# flux pour les quantités de mouvements
def fluxp(th1, th2, dth1, dth2):
    c1 = -(m1+m2)*g*r1*np.sin(th1)-m2*r1*r2*dth1*dth2*np.sin(th1-th2)
    c2 = m2*r1*r2*dth1*dth2*np.sin(th1-th2)-m2*g*r2*np.sin(th2)
    return c1, c2 


# Schéma d'Euler explicite
def euler(th10, th20, p10, p20, dt=0.01, T=10):
    N = int(T/dt)
    temps = np.linspace(0, T, N+1)
    sol = np.zeros((4,N+1))
    sol[0,0] = th10
    sol[1,0] = th20
    sol[2,0] = p10
    sol[3,0] = p20
    for i in range(N):
        a, b = fluxth(sol[0,i], sol[1,i], sol[2,i], sol[3,i])
        sol[0,i+1] = sol[0,i] + dt*a
        sol[1,i+1] = sol[1,i] + dt*b
        c, d = fluxp(sol[0,i], sol[1,i], a, b)
        sol[2,i+1] = sol[2,i] + dt*c
        sol[3,i+1] = sol[3,i] + dt*d
    return temps, sol

def visualisation(temps, sol):
    fig, ax = plt.subplots()
    x1 = r1*np.sin(sol[0,:])
    y1 = -r1*np.cos(sol[0,:])
    x2 = x1 + r2*np.sin(sol[1,:])
    y2 = y1 - r2*np.cos(sol[1,:])
    ax.set_xlim(-(r1+r2), r1+r2)
    ax.set_ylim(-(r1+r2), r1+r2)
    line, = plt.plot([0,x1[0], x2[0]], [0, y1[0], y2[0]], linewidth=2, marker="o", color="blue")
    
    # Trajectoire du pendule
    traj, = plt.plot(x2[:1], y2[:1], color='red', alpha=0.5)

    def maj(i):
        line.set_data([0,x1[i], x2[i]], [0, y1[i], y2[i]])
        traj.set_data(x2[:i], y2[:i])  # Mettre à jour la trajectoire jusqu'à l'indice i
        return line, traj

    ani = animation.FuncAnimation(fig, maj, frames=range(np.size(x1)), blit=True, interval=50)
    return ani

# application

temps, sol = euler(np.pi/2, np.pi/4, 1.5, 0, 0.05, 20)
ani = visualisation(temps, sol)

from IPython.display import HTML
HTML(ani.to_jshtml())

ModuleNotFoundError: No module named 'numpy'